# Reddit Climate Change - Data Preparation
Supervision: Prof. Dr. Jan Fabian Ehmke

Group members: Britz Luis, Huber Anja, Krause Felix Elias, Preda Yvonne-Nadine

Time: Summer term 2023 

Data: https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset

In [1]:
# Loading packages
import pandas as pd
import matplotlib.pyplot as plt

## Load data and pre-processing

In [2]:
# Loading data
raw_comments = pd.read_csv('data/the-reddit-climate-change-dataset-comments.csv')
raw_posts = pd.read_csv('data/the-reddit-climate-change-dataset-posts.csv')

In [3]:
print(raw_comments.shape)
print(raw_posts.shape)

(4600698, 10)
(620908, 12)


In [6]:
raw_comments.columns

Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'body', 'sentiment', 'score', 'post_id'],
      dtype='object')

In [4]:
raw_posts.columns

Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'domain', 'url', 'selftext', 'title',
       'score'],
      dtype='object')

In [5]:
raw_comments["post_id"] = [i.split("/")[6] for i in raw_comments.permalink]

comments_post_id_set = set(raw_comments.post_id)
post_post_id_set = set(raw_posts.id)

len(comments_post_id_set & post_post_id_set)
# For 111.482 (of 620.908) out of all posts there is at least one comment -> approx. 18%

111482